In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
df = pd.read_csv('master.csv').rename(columns={'suicides/100k pop' : 'suicides_per_100k',
                       ' gdp_for_year ($) ' : 'gdp_year', 
                       'gdp_per_capita ($)' : 'gdp_capita',
                       'country-year' : 'country_year'}).assign(gdp_year=lambda _df: _df['gdp_year'].str.replace(',','').astype(np.int64))

In [54]:
df.shape

(27820, 12)

In [7]:
df.head()

,country,year,sex,age,suicides_no,population,suicides_per_100k,country_year,HDI for year,gdp_year,gdp_capita,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


In [8]:
df.country.nunique()

101

In [9]:
print(df.year.nunique(), df.year.max(), df.year.min())

32 2016 1985


In [15]:
df.sex.unique()

array(['male', 'female'], dtype=object)

In [16]:
df.age.unique()

array(['15-24 years', '35-54 years', '75+ years', '25-34 years',
       '55-74 years', '5-14 years'], dtype=object)

In [14]:
df.generation.unique()

array(['Generation X', 'Silent', 'G.I. Generation', 'Boomers',
       'Millenials', 'Generation Z'], dtype=object)

In [18]:
df.describe(include='all')

,country,year,sex,age,suicides_no,population,suicides_per_100k,country_year,HDI for year,gdp_year,gdp_capita,generation
count,27820,27820.000000,27820,27820,27820.000000,2.782000e+04,27820.000000,27820,8364.000000,2.782000e+04,27820.000000,27820
unique,101,NaN,2,6,NaN,NaN,NaN,2321,NaN,NaN,NaN,6
top,Austria,NaN,female,35-54 years,NaN,NaN,NaN,Bahrain1987,NaN,NaN,NaN,Generation X
freq,382,NaN,13910,4642,NaN,NaN,NaN,12,NaN,NaN,NaN,6408
mean,NaN,2001.258375,NaN,NaN,242.574407,1.844794e+06,12.816097,NaN,0.776601,4.455810e+11,16866.464414,NaN
std,NaN,8.469055,NaN,NaN,902.047917,3.911779e+06,18.961511,NaN,0.093367,1.453610e+12,18887.576472,NaN
min,NaN,1985.000000,NaN,NaN,0.000000,2.780000e+02,0.000000,NaN,0.483000,4.691962e+07,251.000000,NaN
25%,NaN,1995.000000,NaN,NaN,3.000000,9.749850e+04,0.920000,NaN,0.713000,8.985353e+09,3447.000000,NaN
50%,NaN,2002.000000,NaN,NaN,25.000000,4.301500e+05,5.990000,NaN,0.779000,4.811469e+10,9372.000000,NaN
75%,NaN,2008.000000,NaN,NaN,131.000000,1.486143e+06,16.620000,NaN,0.855000,2.602024e+11,24874.000000,NaN


In [19]:
def mem_usage(df: pd.DataFrame) -> str:
    """This method styles the memory usage of a DataFrame to be readable as MB.
    Parameters
    ----------
    df: pd.DataFrame
        Data frame to measure.
    Returns
    -------
    str
        Complete memory usage as a string formatted for MB.
    """
    return f'{df.memory_usage(deep=True).sum() / 1024 ** 2 : 3.2f} MB'


def convert_df(df: pd.DataFrame, deep_copy: bool = True) -> pd.DataFrame:
    """Automatically converts columns that are worth stored as
    ``categorical`` dtype.
    Parameters
    ----------
    df: pd.DataFrame
        Data frame to convert.
    deep_copy: bool
        Whether or not to perform a deep copy of the original data frame.
    Returns
    -------
    pd.DataFrame
        Optimized copy of the input data frame.
    """
    return df.copy(deep=deep_copy).astype({
        col: 'category' for col in df.columns
        if df[col].nunique() / df[col].shape[0] < 0.5})

In [20]:
mem_usage(df)

' 10.28 MB'

In [21]:
mem_usage(df.set_index(['country', 'year', 'sex', 'age']))

' 5.00 MB'

In [22]:
mem_usage(convert_df(df))

' 1.40 MB'

In [23]:
mem_usage(convert_df(df.set_index(['country', 'year', 'sex', 'age'])))

' 1.40 MB'

In [24]:
df.query('country == "Albania" and year == 1987 and sex == "male" and age == "25-34 years"')

,country,year,sex,age,suicides_no,population,suicides_per_100k,country_year,HDI for year,gdp_year,gdp_capita,generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


In [61]:
df.loc[4, :]

country                  Albania
year                        1987
sex                         male
age                  25-34 years
suicides_no                    9
population                274300
suicides_per_100k           3.28
country_year         Albania1987
HDI for year                 NaN
gdp_year              2156624900
gdp_capita                   796
generation               Boomers
Name: 4, dtype: object

In [ ]:
#Inefficient technique
df = pd.DataFrame({'a_column': [1, -999, -999],
                    'powerless_column': [2, 3, 4],
                    'int_column': [1, 1, -1]})
df['a_column'] = df['a_column'].replace(-999, np.nan)
df['power_column'] = df['powerless_column'] ** 2
df['real_column'] = df['int_column'].astype(np.float64)
df = df.apply(lambda _df: _df.replace(4, np.nan))
df = df.dropna(how='all')

In [64]:
#Efficient technique
df1 = (pd.DataFrame({'a_column': [1, -999, -999],
                    'powerless_column': [2, 3, 4],
                    'int_column': [1, 1, -1]})
        .assign(a_column=lambda _df: _df['a_column'].replace(-999, np.nan),
                power_column=lambda _df: _df['powerless_column'] ** 2,
                real_column=lambda _df: _df['int_column'].astype(np.float64))
        .apply(lambda _df: _df.replace(4, np.nan))
        .dropna(how='all')
      )

In [65]:
df1

,a_column,powerless_column,int_column,power_column,real_column
0,1.0,2.0,1,NaN,1.0
1,NaN,3.0,1,9.0,1.0
2,NaN,NaN,-1,16.0,-1.0


## Tool-box of method chainers:
- apply  
- assign 
- loc 
- query 
- pipe 
- groupby 
- agg

In [31]:
df.groupby('age').agg({'generation':'unique'}).rename(columns={'generation':'unique_generation'})

,unique_generation
age,
15-24 years,"[Generation X, Millenials]"
25-34 years,"[Boomers, Generation X, Millenials]"
35-54 years,"[Silent, Boomers, Generation X]"
5-14 years,"[Generation X, Millenials, Generation Z]"
55-74 years,"[G.I. Generation, Silent, Boomers]"
75+ years,"[G.I. Generation, Silent]"


In [42]:
(df
 .groupby(['country', 'year'])
 .agg({'suicides_per_100k': 'sum'})
 .rename(columns={'suicides_per_100k':'suicides_sum'})
 .sort_values('suicides_sum', ascending=False)
 .head(10))

suicides_sum
country            year              
Lithuania          1995        639.30
                   1996        595.61
Hungary            1991        575.00
Lithuania          2000        571.80
Hungary            1992        570.26
Lithuania          2001        568.98
Russian Federation 1994        567.64
Lithuania          1998        566.36
                   1997        565.44
                   1999        561.53

In [44]:
(df
 .groupby(['country', 'year'])
 .agg({'suicides_per_100k': 'sum'})
 .rename(columns={'suicides_per_100k':'suicides_sum'})
 .nlargest(10, columns='suicides_sum'))

suicides_sum
country            year              
Lithuania          1995        639.30
                   1996        595.61
Hungary            1991        575.00
Lithuania          2000        571.80
Hungary            1992        570.26
Lithuania          2001        568.98
Russian Federation 1994        567.64
Lithuania          1998        566.36
                   1997        565.44
                   1999        561.53

## Time it

In [52]:
%%time  
for row in df.iterrows(): continue
for tup in df.intertuples():continue

UsageError: Can't use statement directly after '%%time'!
